In [4]:

import pandas as pd
import numpy as np
import seaborn as sns
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession,functions

In [3]:
#!pip install pyspark
!pip install seaborn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 12.7 MB/s eta 0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [7]:
# Create Spark Session
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

23/02/13 21:13:50 WARN Utils: Your hostname, MacdeMacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.48 instead (on interface en0)
23/02/13 21:13:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/13 21:13:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# Read the data into Spark dataframe
DATA_PATH =  'data/'
df = spark.read.csv(DATA_PATH +'transactions_dataset.csv', header = True, inferSchema = True, sep = ';')

In [9]:
# Check the length of the dataset
df.count()

63319315

In [10]:
# Compute the number of total orders by client
df_count = df.groupBy("client_id").count()

In [11]:
#Delete clients who just shopped once
frequent_clients = df_count.filter(df_count['count'] > 1)
frequent_clients.count()

164279

In [12]:
# Get the new filtered dataset with frequent clients, who shopped more than once
filtered_df = df.join(frequent_clients, "client_id", "inner")
filtered_df.show(20)

+---------+-------------------+-------------------+----------+------------------+--------+-------------+---------+-----+
|client_id|         date_order|       date_invoice|product_id|         sales_net|quantity|order_channel|branch_id|count|
+---------+-------------------+-------------------+----------+------------------+--------+-------------+---------+-----+
|      481|2019-01-31 00:00:00|2019-01-31 00:00:00|    915937| 577.5092999999999|       9| at the store|     4952|   42|
|      481|2019-01-31 00:00:00|2019-01-31 00:00:00|    897228|         51.259824|     201| at the store|     4952|   42|
|      481|2019-01-31 00:00:00|2019-01-31 00:00:00|   3139487|          324.4932|       3| at the store|     4952|   42|
|      481|2019-01-31 00:00:00|2019-01-31 00:00:00|   2838301|          27.70856|      11| at the store|     4952|   42|
|      481|2019-02-27 00:00:00|2019-02-27 00:00:00|   1138927| 64.96966400000001|     101| at the store|     3013|   42|
|      481|2019-02-27 00:00:00|2

In [13]:
# Split the date column into year, month and date
split_col = functions.split(filtered_df['date_order'], '-')
filtered_df = filtered_df.withColumn('year', split_col.getItem(0))
filtered_df = filtered_df.withColumn('month', split_col.getItem(1))
filtered_df = filtered_df.withColumn('date', split_col.getItem(2))

In [14]:
# Compute the sales per client by year and month
sales = filtered_df.groupBy("client_id","year","month").sum("sales_net")
sales.show(20)

+---------+----+-----+------------------+
|client_id|year|month|    sum(sales_net)|
+---------+----+-----+------------------+
|      211|2017|   12| 9.522000000000002|
|      211|2018|   03|25.588266666666662|
|      211|2018|   04|196.70520000000002|
|      211|2018|   05|            42.251|
|      211|2018|   06| 654.4236000000001|
|      211|2018|   07|          179.0136|
|      211|2018|   08|          353.9148|
|      211|2018|   09|1121.5827333333334|
|      211|2018|   10|1476.1487706666665|
|      211|2018|   11| 353.2691133333334|
|      211|2018|   12|         215.45664|
|      211|2019|   04|          131.6244|
|      211|2019|   05|407.84096800000003|
|      211|2019|   08| 844.7716000000001|
|      211|2019|   09|2.5115999999999996|
|      481|2019|   01|        980.970884|
|      481|2019|   02|       1107.657184|
|      481|2019|   03|          331.0068|
|      481|2019|   05| 996.3527320000004|
|      481|2019|   08|23.484840000000002|
+---------+----+-----+------------

In [15]:
# Convert this smaller dataset into pandas dataframe for further operations
pd_sales = sales.toPandas()

23/02/13 21:29:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/13 21:29:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:29:50 WARN RowBasedKeyValueBatch: Calling spill() on

23/02/13 21:30:08 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/13 21:30:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:11 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:11 WARN RowBasedKeyValueBatch: Calling spill() on

23/02/13 21:30:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/13 21:30:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/13 21:30:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:34 WARN RowBasedKeyValueBatch: Calling spill() on

23/02/13 21:30:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/13 21:30:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/13 21:30:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/13 21:30:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [16]:
pd_sales.to_csv('sales_by_month.csv')